In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
lang = pd.read_csv('sentences.csv',sep='\t')
lang = lang.dropna()
lang.head()

,1,cmn,我們試試看！
0,2,cmn,我该去睡觉了。
1,3,cmn,你在干什麼啊？
2,4,cmn,這是什麼啊？
3,5,cmn,今天是６月１８号，也是Muiriel的生日！
4,6,cmn,生日快乐，Muiriel！


In [3]:
X, Y = [], []
for no, ln in enumerate(lang.cmn.unique()):
    langs = lang.loc[lang.cmn == ln]
    if langs.shape[0] < 500:
        continue
    print(no, ln)
    langs = langs.iloc[:500,-1].tolist()
    X.extend(langs)
    Y.extend([ln] * len(langs))

0 cmn
1 deu
2 rus
3 fra
4 eng
5 jpn
6 spa
7 ita
8 kor
9 vie
10 nld
11 epo
12 tur
13 por
14 heb
15 hun
16 ell
17 ind
18 ara
19 arz
20 fin
21 bul
22 yue
23 swe
24 ukr
25 bel
27 ces
29 nno
30 wuu
31 nob
32 zsm
33 est
34 kat
35 pol
36 lat
37 urd
38 sqi
39 isl
41 afr
42 ron
45 bre
46 tat
47 yid
48 uig
49 uzb
50 srp
52 dan
53 pes
54 slk
55 eus
58 tgl
59 lvs
60 kaz
61 hye
62 hin
63 lit
64 ben
65 cat
67 hrv
68 tha
69 orv
71 mon
72 lzh
74 gle
75 mkd
76 slv
78 glg
79 vol
81 jbo
82 toki
83 ina
84 nds
85 mal
86 tlh
90 ido
91 gla
96 oci
97 ile
98 xal
102 khm
105 aze
108 chv
109 hsb
110 dsb
113 cym
116 kur
118 kab
119 ber
120 tpw
123 pms
125 grn
129 grc
133 cor
135 avk
136 mar
141 prg
148 cbk
153 arq
157 rom
166 mhr
196 sah
222 kha
223 ceb
228 lfn
236 kzj
237 pam
246 zza
254 dtp
260 ilo
264 hoc
268 war
273 csb
289 nst


In [4]:
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

def clean_text(string):
    string = re.sub(u'[0-9!@#$%^&*()_\-+{}|\~`\'";:?/.>,<]', ' ', string.lower(), flags=re.UNICODE)
    return re.sub(r'[ ]+', ' ', string.lower()).strip()

In [5]:
X =[clean_text(s) for s in X]

In [6]:
bow_chars = CountVectorizer(ngram_range=(3, 5), analyzer='char_wb', max_features=700000).fit(X)
delattr(bow_chars, 'stop_words_')
target = LabelEncoder().fit_transform(Y)
features = bow_chars.transform(X)
features.shape

(56500, 688217)

In [7]:
train_X, test_X, train_Y, test_Y = train_test_split(features, target, test_size = 0.2)
del features

In [8]:
from sklearn import metrics

In [9]:
def convert_sparse_matrix_to_sparse_tensor(X, limit = 5):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    coo.data[coo.data > limit] = limit
    return tf.SparseTensorValue(indices, coo.col, coo.shape), tf.SparseTensorValue(indices, coo.data, coo.shape)

In [10]:
labels = np.unique(Y,return_counts=True)[0]
labels

array(['afr', 'ara', 'arq', 'arz', 'avk', 'aze', 'bel', 'ben', 'ber',
       'bre', 'bul', 'cat', 'cbk', 'ceb', 'ces', 'chv', 'cmn', 'cor',
       'csb', 'cym', 'dan', 'deu', 'dsb', 'dtp', 'ell', 'eng', 'epo',
       'est', 'eus', 'fin', 'fra', 'gla', 'gle', 'glg', 'grc', 'grn',
       'heb', 'hin', 'hoc', 'hrv', 'hsb', 'hun', 'hye', 'ido', 'ile',
       'ilo', 'ina', 'ind', 'isl', 'ita', 'jbo', 'jpn', 'kab', 'kat',
       'kaz', 'kha', 'khm', 'kor', 'kur', 'kzj', 'lat', 'lfn', 'lit',
       'lvs', 'lzh', 'mal', 'mar', 'mhr', 'mkd', 'mon', 'nds', 'nld',
       'nno', 'nob', 'nst', 'oci', 'orv', 'pam', 'pes', 'pms', 'pol',
       'por', 'prg', 'rom', 'ron', 'rus', 'sah', 'slk', 'slv', 'spa',
       'sqi', 'srp', 'swe', 'tat', 'tgl', 'tha', 'tlh', 'toki', 'tpw',
       'tur', 'uig', 'ukr', 'urd', 'uzb', 'vie', 'vol', 'war', 'wuu',
       'xal', 'yid', 'yue', 'zsm', 'zza'], dtype='<U4')

In [11]:
class Model:
    def __init__(self, learning_rate):
        self.X = tf.sparse_placeholder(tf.int32)
        self.W = tf.sparse_placeholder(tf.int32)
        self.Y = tf.placeholder(tf.int32, [None])
        embeddings = tf.Variable(tf.truncated_normal([train_X.shape[1], 64]))
        embed = tf.nn.embedding_lookup_sparse(embeddings, self.X, self.W, combiner='mean')
        self.logits = tf.layers.dense(embed, len(labels))
        self.cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits = self.logits, labels = self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        correct_pred = tf.equal(tf.argmax(self.logits, 1,output_type=tf.int32), self.Y)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [12]:
sess = tf.InteractiveSession()
model = Model(1e-4)
sess.run(tf.global_variables_initializer())

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [13]:
import time
from tqdm import tqdm

batch_size = 64
for e in range(50):
    lasttime = time.time()
    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    pbar = tqdm(
        range(0, train_X.shape[0], batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        batch_x = convert_sparse_matrix_to_sparse_tensor(train_X[i : min(i + batch_size, train_X.shape[0])])
        batch_y = train_Y[i : min(i + batch_size, train_X.shape[0])]
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.X: batch_x[0],
                model.W: batch_x[1],
                model.Y: batch_y
            },
        )
        assert not np.isnan(cost)
        train_loss += cost
        train_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    pbar = tqdm(range(0, test_X.shape[0], batch_size), desc = 'test minibatch loop')
    for i in pbar:
        batch_x = convert_sparse_matrix_to_sparse_tensor(test_X[i : min(i + batch_size, test_X.shape[0])])
        batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
        batch_x_expand = np.expand_dims(batch_x,axis = 1)
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.X: batch_x[0],
                model.W: batch_x[1],
                model.Y: batch_y
            },
        )
        test_loss += cost
        test_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    train_loss /= train_X.shape[0] / batch_size
    train_acc /= train_X.shape[0] / batch_size
    test_loss /= test_X.shape[0] / batch_size
    test_acc /= test_X.shape[0] / batch_size

    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (e, train_loss, train_acc, test_loss, test_acc)
    )

train minibatch loop:   0%|          | 3/707 [00:00<00:32, 21.71it/s, accuracy=0.0625, cost=4.68]

time taken: 33.243767976760864
epoch: 0, training loss: 4.724888, training acc: 0.013584, valid loss: 4.713782, valid acc: 0.021131



train minibatch loop:   0%|          | 3/707 [00:00<00:32, 21.58it/s, accuracy=0.0938, cost=4.64]

time taken: 33.123547315597534
epoch: 1, training loss: 4.684196, training acc: 0.036858, valid loss: 4.676456, valid acc: 0.050777



train minibatch loop:   0%|          | 3/707 [00:00<00:32, 21.63it/s, accuracy=0.125, cost=4.59]

...

time taken: 33.111047983169556
epoch: 47, training loss: 0.286186, training acc: 0.973208, valid loss: 0.729391, valid acc: 0.874415



train minibatch loop:   0%|          | 3/707 [00:00<00:32, 21.75it/s, accuracy=0.984, cost=0.174]

time taken: 33.11710834503174
epoch: 48, training loss: 0.267859, training acc: 0.974624, valid loss: 0.710492, valid acc: 0.875477



test minibatch loop: 100%|██████████| 177/177 [00:00<00:00, 408.73it/s, accuracy=0.889, cost=0.756]

time taken: 33.10894179344177
epoch: 49, training loss: 0.250987, training acc: 0.976394, valid loss: 0.692780, valid acc: 0.877778



In [14]:
real_Y, predict_Y = [], []

pbar = tqdm(
    range(0, test_X.shape[0], batch_size), desc = 'validation minibatch loop'
)
for i in pbar:
    batch_x = convert_sparse_matrix_to_sparse_tensor(test_X[i : min(i + batch_size, test_X.shape[0])])
    batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])].tolist()
    predict_Y += np.argmax(
        sess.run(
            model.logits, feed_dict = {model.X: batch_x[0], model.W: batch_x[1], model.Y: batch_y}
        ),
        1,
    ).tolist()
    real_Y += batch_y

validation minibatch loop: 100%|██████████| 177/177 [00:00<00:00, 759.90it/s]


In [15]:
print(
    metrics.classification_report(
        real_Y, predict_Y, target_names = labels
    )
)

              precision    recall  f1-score   support

         afr       0.92      0.92      0.92        88
         ara       0.86      0.84      0.85       114
         arq       0.88      0.88      0.88       105
         arz       0.72      0.74      0.73        98
         avk       0.93      0.87      0.90       105
         aze       0.92      0.92      0.92       104
         bel       0.91      0.89      0.90       111
         ben       1.00      1.00      1.00       108
         ber       0.88      0.86      0.87       103
         bre       0.94      0.96      0.95       102
         bul       0.79      0.88      0.83       113
         cat       0.79      0.82      0.80       116
         cbk       0.99      0.92      0.95       115
         ceb       0.83      0.80      0.81       108
         ces       0.79      0.75      0.77       111
         chv       0.97      0.94      0.96       109
         cmn       0.47      0.19      0.27        99
         cor       0.93    